In [24]:
import openai
from IPython.display import display
import clip
from PIL import Image as PILImage
import torch
import os
from io import BytesIO
from IPython.display import Image as DisplayImage


# Set the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = "INSERT OPEN AI API KEY"

# Initialize the OpenAI client with the API key
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Load the model
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    return model, preprocess, device

# Generate and display an image based on the input and description
def generate_image(input_image_path, description):
    model, preprocess, device = load_model()

    # Load and preprocess the image
    original_image = PILImage.open(input_image_path)
    image = preprocess(original_image).unsqueeze(0).to(device)

    # Tokenize and encode text and image
    with torch.no_grad():
        image_features = model.encode_image(image)
        

    # Assuming you have a method to create a detailed prompt based on the original image features
    detailed_prompt = f"Create a super realistic render of the modifications based on the original image features, making it an edit off the original house: {description}"

    # Call OpenAI API to generate the image
    response = client.images.generate(
        prompt=detailed_prompt,
        n=1,
        size="512x512"
    )

    # Extract the image URL from the response and display it
    if response.data:
        image_url = response.data[0].url
        display(DisplayImage(url=image_url))
    else:
        print("No image data found in response")

# Example usage
input_image_path = "data/testimage.png"
description = "Imagine the house with a larger second floor and a new garden area."
generate_image(input_image_path, description)


BadRequestError: Error code: 400 - {'error': {'code': None, 'message': "'Create a super realistic render of the modifications based on the original image features, making it an edit off the original house: Imagine the house with a larger second floor and a new garden area.tensor([[-6.0605e-02,  1.7822e-01,  2.3539e-01, -2.2298e-01,  2.3070e-01,\\n         -7.8016e-02,  4.2728e-01,  2.4324e-01,  6.8957e-01, -1.7371e-01,\\n          7.8337e-02,  8.3102e-02,  4.4959e-01, -2.1086e-02, -1.6960e-01,\\n         -4.5775e-02, -6.4439e-01, -1.3164e-01, -9.4267e-02,  5.3512e-01,\\n          6.9133e-01, -2.3160e-02, -1.5675e-01,  5.3444e-01,  3.4615e-01,\\n         -9.7210e-02,  1.1851e-01, -1.7795e-01,  4.0064e-02,  3.8034e-01,\\n          1.5928e-01,  2.0482e-02, -2.0989e-01, -2.9203e-01,  1.0622e-01,\\n          6.8663e-01, -8.3527e-04,  1.0842e-01,  2.1795e-01,  1.4935e+00,\\n         -2.7752e-01, -1.0983e-02, -4.6198e-01, -7.1679e-01,  3.3953e-01,\\n         -1.2593e+00,  7.0536e-02,  1.2301e

In [ ]:
import torch
import requests
from PIL import Image
from diffusers import StableDiffusionDepth2ImgPipeline

pipe = StableDiffusionDepth2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-depth",
    torch_dtype=torch.float16,
).to("cuda")



img = load_image("data/testimage.png")
prompt = "Add a second floor to this image"
pipe(prompt=prompt, image=img, negative_prompt=None, strength=0.7).images[0]


In [5]:
!pip uninstall openai
!pip install openai

Found existing installation: openai 1.16.2
Uninstalling openai-1.16.2:
  Would remove:
    /Users/yashgupta/opt/anaconda3/bin/openai
    /Users/yashgupta/opt/anaconda3/lib/python3.9/site-packages/openai-1.16.2.dist-info/*
    /Users/yashgupta/opt/anaconda3/lib/python3.9/site-packages/openai/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [11]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
# from dotenv import load_dotenv, find_dotenv
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
import openai
import os

os.environ["OPENAI_API_KEY"] = "INSERT OPEN AI API KEY"


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)
image = encode_image("data/testimage.png")


msg = chain.invoke(
    [   AIMessage(
        content="You are an observant bot that recognizes every little detail of a house."
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "Identify all features of the house in this image and provide an extremely detailed description or list of features the house includes."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)

print(msg.content)


chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)
prompt = PromptTemplate.from_template(
"""
You see the following features in a house:
{features}
Modify the house based on the specifications below and describe the house in very detailed format to where it can be recreated. 

Output:
"""
)
runnable = prompt | chain | StrOutputParser()

description = "Add another floor to the house"

house = runnable.invoke({"features": msg.content})
print(house)

client = openai.OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt=description,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
print(image_url)

AttributeError: module 'openai' has no attribute 'OpenAI'